In [1]:
# mike babb
# 2024 06 28
# what streets start and stop?
# step 01: import data / perform some initial formatting

In [2]:
# standard
import os

In [3]:
# external
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import shapely

# load the streent network shapefile as a gpd

In [4]:
# file paths
input_file_path = 'H:/project/seattle_streets/data/' 
output_file_path = 'H:/project/seattle_streets/data/'

In [5]:
file_name = 'Street_Network_Database_SND_5117857036965774451.gpkg'

In [6]:
fpn = os.path.join(input_file_path, file_name)

In [7]:
gdf = gpd.read_file(filename = fpn)

C:\Users\babbm\anaconda3\envs\analysis\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column SNDSEG_UPDATE, 2007-04-22T00:00:00.0Z, successfully parsed
  return ogr_read(


# LOWER CASE AND RENAME COLUMNS

In [8]:
gdf.columns = [cn.lower() for cn in gdf.columns]

In [9]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'geometry'],
      dtype='object')

# ADD A COLUMN INDICATING WHICH PORTION OF THE CITY THE STREET IS IN

In [10]:
# combine the ord_pre_dir and the ord_suf_dir to get the portion of the city.

In [11]:
col_names = ['ord_pre_dir', 'ord_suf_dir']

In [12]:
for cn in col_names:
    gdf[cn] = gdf[cn].fillna('')

In [13]:
def create_city_portion(my_row:pd.Series):
    
    opd = my_row['ord_pre_dir']
    osd = my_row['ord_suf_dir']

    outcome = opd + ' ' + osd
    outcome = outcome.strip()
    if outcome == '':
        outcome = ' '    
    
    return outcome

In [14]:
gdf['city_portion'] = gdf.apply(create_city_portion, axis = 1)

In [15]:
gdf['city_portion'].unique()

array(['S', 'E', 'NE', 'NW', 'N', ' ', 'SW', 'W', 'WB', 'EB', 'SB', 'NB'],
      dtype=object)

# WRITE THE GEODATAFRAME TO DISK AS A GPKG

In [16]:
# file path
output_file_name = 'Street_Network_Database_Complete.gpkg'

In [17]:
ofpn = os.path.join(output_file_path, output_file_name)

In [18]:
# reproject to wgs 84
gdf = gdf.to_crs(crs = 'epsg:4326')

In [19]:
gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)

# EXPORT THE BLANK STREET TYPES TO AN EXCEL FILE FOR MANUAL INSPECTION

In [20]:
gdf['ord_street_type'].unique()

array(['AVE', 'CT', 'ST', 'PL', 'LN', 'RD', 'DR', 'WAY', 'PKWY', 'BLVD',
       'CIR', 'TER', None, 'AL', 'RP', 'LOOP', 'OP', 'TRL', 'BR', 'ET',
       'VIEW', 'HWY', 'PZ', 'MALL'], dtype=object)

In [21]:
blank_street_type_df = gdf.loc[gdf['ord_street_type'].isna(), 'ord_stname_concat'].drop_duplicates().to_frame()

In [22]:
blank_street_type_df.shape

(80, 1)

In [23]:
output_file_name = 'blank_street_type.xlsx'

In [24]:
ofpn = os.path.join(output_file_path, output_file_name)

In [25]:
blank_street_type_df.to_excel(excel_writer=ofpn, index = False)